In [2]:
from ultralytics import YOLO
import os, csv, cv2

In [2]:
person_model = YOLO("weight/yolov8s.pt")
face_model = YOLO("weight/yolov8n-face-lindevs.pt")
head_model = YOLO("weight/nano.pt")

In [3]:
INPUT_VIDEO = "input/input_video.mp4"
CONFIDENCE = 0.35
IMGSZ = 960
SAVE_DIR = "out/"

In [4]:
OUT = f"{SAVE_DIR}detection.mp4"
CSV = f"{SAVE_DIR}count.csv"
cap = cv2.VideoCapture(INPUT_VIDEO)
w, h = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS) or 30
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(OUT, fourcc, fps, (w,h))

In [5]:
def to_list(result):
    return result.boxes.xyxy.cpu().numpy().astype(int).tolist(), result.boxes.conf.cpu().numpy().tolist()

In [ ]:
with open(CSV, "w", newline = "") as f:
    wr = csv.writer(f)
    wr.writerow(["frame", "n_person", "n_face", "n_head"])
    frame = 0
    while True:
        ok, img = cap.read()
        if not ok: break
        frame += 1
        r_p = person_model.predict(img, classes = [0], conf = CONFIDENCE, imgsz = IMGSZ, verbose = False)[0]
        r_h = head_model.predict(img, conf = CONFIDENCE, imgsz = IMGSZ, verbose = False)[0]
        r_f = face_model.predict(img, conf = CONFIDENCE, imgsz = IMGSZ, verbose = False)[0]

        P, Pc = to_list(r_p)
        F, Fc = to_list(r_f)
        H, Hc = to_list(r_h)
        
        for x1, y1, x2, y2 in P: cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
        for x1, y1, x2, y2 in F: cv2.rectangle(img, (x1,y1), (x2,y2), (255,0,0), 2)
        for x1, y1, x2, y2 in H: cv2.rectangle(img, (x1,y1), (x2,y2), (255,0,255), 2)
        
        wr.writerow([frame, len(P), len(F), len(H)])
        writer.write(img)
        
cap.release(); writer.release()
print("Saved:", OUT, "CSV:", CSV)
        

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Saved: out/detection.mp4 CSV: out/count.csv


In [ ]:
model = YOLO("yolo11n.pt")
results = model.track(source=INPUT_VIDEO, conf=0.3, iou=0.5, show=True, tracker = "tracker.yaml")



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs



c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


video 1/1 (frame 1/341) c:\Users\ADMIN\Desktop\D-Soft Project\Tun 4\Code_Retail_Analytics\input\input_video.mp4: 384x640 16 persons, 260.8ms
video 1/1 (frame 2/341) c:\Users\ADMIN\Desktop\D-Soft Project\Tun 4\Code_Retail_Analytics\input\input_video.mp4: 384x640 14 persons, 169.7ms
video 1/1 (frame 3/341) c:\Users\ADMIN\Desktop\D-Soft Project\Tun 4\Code_Retail_Analytics\input\input_video.mp4: 384x640 15 persons, 146.1ms
video 1/1 (frame 4/341) c:\Users\ADMIN\Desktop\D-Soft Project\Tun 4\Code_Retail_Analytics\input\input_video.mp4: 384x640 17 persons, 127.3ms
video 1/1 (frame 5/341) c:\Users\ADMIN\Desktop\D-Soft Project\Tun 4\Code_Retail_Analytics\input\input_video.mp4: 384x640 17 persons, 134.3ms
video 1/1 (frame 6/341) c:\Users\ADMIN\Desktop\D-Soft Project\Tun 4\Code_Retail_Analytics\input\input_video.mp4: 384x640 17 persons, 144.1ms
video 1/1 (frame 7/341) c:\Users\ADMIN\Desktop\D-Soft Project\Tun 4\Code_Retail_Analytics\input\input_video.mp4: 384x640 17 persons, 130.6ms
video 1/1 (fr

: 